# Naive representation pipeline

Main steps:

* input: image of the sky (frequency domain)
* apply 2D Fourier transform --> visibilities (Fourier domain)
* encode classical visibilities data into qbits (non-conventional domain)
* measure qbits (back to conventional domain)
* apply 2D Inverse Fourier transform --> original image?

In [46]:
import numpy as np
import sys
import qiskit
from qiskit import *
from qiskit.visualization import plot_histogram
import struct

## Generating an artificial image of the sky

In [47]:
n = 4
#setting very low complex random values (image background) - single precision complex floats
image = np.zeros((n, n), dtype=complex)
image.real = np.random.rand(n , n) / 100
image.imag = np.random.rand(n , n) / 100

#setting few larger complex random values in a specific area (image source/subject)
source_middle = int(n/2)
source_radius = int(n/4)
image[source_middle-source_radius:source_middle+source_radius, source_middle-source_radius:source_middle+source_radius].real = ((np.random.rand(int(n/2), int(n/2)) +100) * 100)
image[source_middle-source_radius:source_middle+source_radius, source_middle-source_radius:source_middle+source_radius].imag = ((np.random.rand(int(n/2), int(n/2)) +100) * 100)

print(image)

[[1.16195438e-03+2.97003258e-03j 9.71314411e-04+1.16207750e-03j
  4.49647779e-03+4.23586972e-03j 9.65850907e-03+5.21884115e-03j]
 [6.18517190e-03+2.54055661e-03j 1.00585221e+04+1.00324746e+04j
  1.00921342e+04+1.00643031e+04j 6.99787129e-03+7.78623404e-03j]
 [6.89846435e-03+6.91549800e-03j 1.00816718e+04+1.00960228e+04j
  1.00453892e+04+1.00377905e+04j 3.82782106e-03+2.48224924e-03j]
 [5.57713909e-03+1.39341030e-03j 6.42264351e-03+5.14206165e-03j
  7.86788081e-03+9.41347893e-03j 2.23740100e-03+3.22370594e-03j]]


## Applying 2D Fourier transform (visibilities)

In [48]:
visibilities = np.fft.fft2(image)
print(visibilities)

[[ 4.02777796e+04+4.02306435e+04j -9.03084355e+00-4.02422720e+04j
  -2.66849571e+00-2.64013921e+01j -4.02660009e+04+3.80851505e+01j]
 [-3.02865766e+01-4.02844565e+04j -2.01734489e+04+2.01791198e+04j
   6.80954151e+01+2.46221763e+01j  2.01356217e+04+2.00806963e+04j]
 [-2.36035262e+01+3.70291201e+01j  1.10292225e+02+3.38362077e+00j
  -6.99006757e+01-9.00527008e+01j -1.68028310e+01+4.96637661e+01j]
 [-4.02238243e+04+1.68381774e+01j  2.00721579e+04+2.00597983e+04j
   4.45387083e+00+9.18352164e+01j  2.01471850e+04-2.01684850e+04j]]


### Sanity check

In [61]:
img = np.fft.ifft2(visibilities)
print(img)
test_real = ((image.real - img.real)**2).mean()
test_imag = ((image.imag - img.imag)**2).mean()

print()
 
print(test_real)
print(test_imag)

[[1.16195438e-03+2.97003258e-03j 9.71314412e-04+1.16207750e-03j
  4.49647780e-03+4.23586972e-03j 9.65850907e-03+5.21884115e-03j]
 [6.18517189e-03+2.54055661e-03j 1.00585221e+04+1.00324746e+04j
  1.00921342e+04+1.00643031e+04j 6.99787129e-03+7.78623404e-03j]
 [6.89846435e-03+6.91549800e-03j 1.00816718e+04+1.00960228e+04j
  1.00453892e+04+1.00377905e+04j 3.82782106e-03+2.48224924e-03j]
 [5.57713909e-03+1.39341030e-03j 6.42264351e-03+5.14206165e-03j
  7.86788081e-03+9.41347894e-03j 2.23740100e-03+3.22370594e-03j]]

6.027991610064458e-25
1.1511156345466998e-24


## Classical data encoding/decoding

In [66]:
number_of_entries = visibilities.shape[0] * visibilities.shape[1]
number_of_bits_real = 32 #since single precision complex floats
number_of_bits_imag = 32
number_of_bits = number_of_bits_real + number_of_bits_imag
off_set = 0

qc = QuantumCircuit(number_of_entries*number_of_bits) #n bits encoded into n qbits (circuit family #1)
for i in range(0, visibilities.shape[0]):
    for j in range(0, visibilities.shape[1]):
            binary_real = float_to_bin_real(visibilities[i, j].real)
            binary_imag = float_to_bin_imag(visibilities[i, j].imag)
            binary = binary_real+binary_imag
            off_set = encoding2(qc, binary, off_set)

#measurement
qc.measure_all()
backend = Aer.get_backend('aer_simulator')
job = backend.run(qc, shots=1, memory=True)
output = job.result().get_memory()[0]
out = reverse(output)

#readout
chunks_real = []
chunks_imag = []
for i in range(0, number_of_entries):
    chunks_real.append(out[number_of_bits*i:(number_of_bits*i)+number_of_bits_real]) #real parts represented every 64 bits starting from the first one
    chunks_imag.append(out[(number_of_bits*i)+number_of_bits_imag:(number_of_bits*i)+number_of_bits_imag+number_of_bits_imag]) #imaginary parts represented every 64 bits starting after the first occurence of a real part  
readout = []
for i in range(0, len(chunks_real)):
    readout.append(complex(bin_to_float_real(chunks_real[i]), bin_to_float_imag(chunks_imag[i]))) 

#reshaping the readout vector into a nxn matrix
readout = np.array(readout).reshape(n , n)
print(readout)

[[ 4.02777812e+04+4.02306445e+04j -9.03084373e+00-4.02422734e+04j
  -2.66849566e+00-2.64013920e+01j -4.02660000e+04+3.80851517e+01j]
 [-3.02865772e+01-4.02844570e+04j -2.01734492e+04+2.01791191e+04j
   6.80954132e+01+2.46221771e+01j  2.01356211e+04+2.00806953e+04j]
 [-2.36035271e+01+3.70291214e+01j  1.10292229e+02+3.38362074e+00j
  -6.99006729e+01-9.00527039e+01j -1.68028316e+01+4.96637650e+01j]
 [-4.02238242e+04+1.68381767e+01j  2.00721582e+04+2.00597988e+04j
   4.45387077e+00+9.18352127e+01j  2.01471855e+04-2.01684844e+04j]]


## Applying 2D Inverse Fourier transform (+ fidelity test)

In [67]:
img = np.fft.ifft2(readout)
print(img)
test_real = ((image.real - img.real)**2).mean()
test_imag = ((image.imag - img.imag)**2).mean()

print()

print(test_real)
print(test_imag)

[[1.32629275e-03+2.88113952e-03j 1.15710497e-03+1.12903118e-03j
  4.55400348e-03+4.38037515e-03j 9.69433784e-03+5.30898571e-03j]
 [6.55981898e-03+2.39744782e-03j 1.00585223e+04+1.00324746e+04j
  1.00921341e+04+1.00643033e+04j 7.08490610e-03+7.92038441e-03j]
 [7.06216693e-03+6.95094466e-03j 1.00816720e+04+1.00960229e+04j
  1.00453892e+04+1.00377907e+04j 3.82542610e-03+2.63249874e-03j]
 [5.52937388e-03+1.48382783e-03j 6.61897659e-03+5.16998768e-03j
  8.05947185e-03+9.45839286e-03j 2.18445063e-03+3.32987309e-03j]]

2.5575246394798197e-08
1.3372947278057183e-08


## Utils

### Quantum

In [68]:
def encoding2(qc, binary, off_set):
    
    for i in range(0, len(binary)):
        qc.reset(off_set+i)

        if binary[i]=='1':
            qc.x(off_set+i)
    
    off_set += len(binary)

    return off_set

### Classical

In [69]:
#float to binary / binary to float
def float_to_bin_real(num):
    return format(struct.unpack('!I', struct.pack('!f', num))[0], '032b')
def float_to_bin_imag(num):
    return format(struct.unpack('!I', struct.pack('!f', num))[0], '032b')

def bin_to_float_real(binary):
    return struct.unpack('!f',struct.pack('!I', int(binary, 2)))[0]
def bin_to_float_imag(binary):
    return struct.unpack('!f',struct.pack('!I', int(binary, 2)))[0]

#reverse a string (for the measurement step readout)
def reverse(string):
    string = string[::-1]
    return string